# Mecanismos de atención y transformadores

Una de las principales desventajas de las redes recurrentes es que todas las palabras en una secuencia tienen el mismo impacto en el resultado. Esto provoca un rendimiento subóptimo en los modelos estándar de codificador-decodificador LSTM para tareas de secuencia a secuencia, como el Reconocimiento de Entidades Nombradas y la Traducción Automática. En realidad, palabras específicas en la secuencia de entrada suelen tener más impacto en las salidas secuenciales que otras.

Consideremos un modelo de secuencia a secuencia, como la traducción automática. Este se implementa mediante dos redes recurrentes, donde una red (**codificador**) colapsa la secuencia de entrada en un estado oculto, y otra (**decodificador**) desenvuelve este estado oculto en el resultado traducido. El problema con este enfoque es que el estado final de la red tiene dificultades para recordar el inicio de una oración, lo que provoca una baja calidad del modelo en oraciones largas.

**Los mecanismos de atención** proporcionan un medio para ponderar el impacto contextual de cada vector de entrada en cada predicción de salida de la RNN. Esto se implementa creando atajos entre los estados intermedios de la RNN de entrada y la RNN de salida. De esta manera, al generar el símbolo de salida $y_t$, tomaremos en cuenta todos los estados ocultos de entrada $h_i$, con diferentes coeficientes de peso $\alpha_{t,i}$. 

![Imagen que muestra un modelo codificador/decodificador con una capa de atención aditiva](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*El modelo codificador-decodificador con mecanismo de atención aditiva en [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citado de [este blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

La matriz de atención $\{\alpha_{i,j}\}$ representaría el grado en que ciertas palabras de entrada influyen en la generación de una palabra dada en la secuencia de salida. A continuación, se muestra un ejemplo de dicha matriz:

![Imagen que muestra un alineamiento de muestra encontrado por RNNsearch-50, tomado de Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Figura tomada de [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Los mecanismos de atención son responsables de gran parte del estado del arte actual o cercano al actual en el procesamiento de lenguaje natural. Sin embargo, agregar atención aumenta significativamente el número de parámetros del modelo, lo que llevó a problemas de escalabilidad con las RNNs. Una restricción clave para escalar las RNNs es que la naturaleza recurrente de los modelos hace que sea difícil agrupar y paralelizar el entrenamiento. En una RNN, cada elemento de una secuencia necesita ser procesado en orden secuencial, lo que significa que no se puede paralelizar fácilmente.

La adopción de mecanismos de atención combinada con esta restricción llevó a la creación de los modelos transformadores, ahora estado del arte, que conocemos y usamos hoy en día, desde BERT hasta OpenGPT3.

## Modelos transformadores

En lugar de transmitir el contexto de cada predicción previa al siguiente paso de evaluación, los **modelos transformadores** utilizan **codificaciones posicionales** y atención para capturar el contexto de una entrada dada dentro de una ventana de texto proporcionada. La imagen a continuación muestra cómo las codificaciones posicionales con atención pueden capturar el contexto dentro de una ventana dada.

![GIF animado que muestra cómo se realizan las evaluaciones en los modelos transformadores.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Dado que cada posición de entrada se mapea de manera independiente a cada posición de salida, los transformadores pueden paralelizar mejor que las RNNs, lo que permite modelos de lenguaje mucho más grandes y expresivos. Cada cabeza de atención puede usarse para aprender diferentes relaciones entre palabras que mejoran las tareas de procesamiento de lenguaje natural.

**BERT** (Representaciones de Codificador Bidireccional de Transformadores) es una red transformadora multinivel muy grande con 12 capas para *BERT-base* y 24 para *BERT-large*. El modelo se preentrena primero en un gran corpus de datos de texto (Wikipedia + libros) utilizando entrenamiento no supervisado (predicción de palabras enmascaradas en una oración). Durante el preentrenamiento, el modelo absorbe un nivel significativo de comprensión del lenguaje que luego puede aprovecharse con otros conjuntos de datos mediante ajuste fino. Este proceso se llama **aprendizaje por transferencia**. 

![Imagen de http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Existen muchas variaciones de arquitecturas de transformadores, incluyendo BERT, DistilBERT, BigBird, OpenGPT3 y más, que pueden ajustarse. El paquete [HuggingFace](https://github.com/huggingface/) proporciona un repositorio para entrenar muchas de estas arquitecturas con PyTorch. 

## Usando BERT para clasificación de texto

Veamos cómo podemos usar el modelo BERT preentrenado para resolver nuestra tarea tradicional: clasificación de secuencias. Clasificaremos nuestro conjunto de datos original AG News.

Primero, carguemos la biblioteca HuggingFace y nuestro conjunto de datos:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Dado que utilizaremos un modelo BERT preentrenado, necesitaremos usar un tokenizador específico. Primero, cargaremos un tokenizador asociado con el modelo BERT preentrenado.

La biblioteca HuggingFace contiene un repositorio de modelos preentrenados, que puedes usar simplemente especificando sus nombres como argumentos en las funciones `from_pretrained`. Todos los archivos binarios necesarios para el modelo se descargarán automáticamente.

Sin embargo, en ciertos casos podrías necesitar cargar tus propios modelos, en cuyo caso puedes especificar el directorio que contiene todos los archivos relevantes, incluidos los parámetros para el tokenizador, el archivo `config.json` con los parámetros del modelo, los pesos binarios, etc.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

El objeto `tokenizer` contiene la función `encode` que puede usarse directamente para codificar texto:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Luego, creemos iteradores que utilizaremos durante el entrenamiento para acceder a los datos. Debido a que BERT utiliza su propia función de codificación, necesitaríamos definir una función de relleno similar a `padify` que hemos definido antes:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

En nuestro caso, usaremos un modelo BERT preentrenado llamado `bert-base-uncased`. Vamos a cargar el modelo utilizando el paquete `BertForSequenceClassfication`. Esto asegura que nuestro modelo ya tenga una arquitectura requerida para la clasificación, incluyendo el clasificador final. Verás un mensaje de advertencia que indica que los pesos del clasificador final no están inicializados y que el modelo requeriría preentrenamiento; eso está perfectamente bien, porque es exactamente lo que estamos a punto de hacer.


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

¡Ahora estamos listos para comenzar el entrenamiento! Dado que BERT ya está preentrenado, queremos empezar con una tasa de aprendizaje bastante pequeña para no alterar los pesos iniciales.

Todo el trabajo pesado lo realiza el modelo `BertForSequenceClassification`. Cuando llamamos al modelo con los datos de entrenamiento, este devuelve tanto la pérdida como la salida de la red para el minibatch de entrada. Usamos la pérdida para la optimización de parámetros (`loss.backward()` realiza el paso hacia atrás) y `out` para calcular la precisión del entrenamiento comparando las etiquetas obtenidas `labs` (calculadas usando `argmax`) con las etiquetas esperadas `labels`.

Para controlar el proceso, acumulamos la pérdida y la precisión a lo largo de varias iteraciones y las imprimimos cada `report_freq` ciclos de entrenamiento.

Este entrenamiento probablemente tomará bastante tiempo, por lo que limitamos el número de iteraciones.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


¡Puedes ver (especialmente si aumentas el número de iteraciones y esperas lo suficiente) que la clasificación con BERT nos da una precisión bastante buena! Esto se debe a que BERT ya comprende bastante bien la estructura del lenguaje, y solo necesitamos ajustar el clasificador final. Sin embargo, debido a que BERT es un modelo grande, todo el proceso de entrenamiento lleva mucho tiempo y requiere una potencia computacional considerable (GPU, y preferiblemente más de una).

> **Note:** En nuestro ejemplo, hemos estado utilizando uno de los modelos BERT preentrenados más pequeños. Existen modelos más grandes que probablemente ofrezcan mejores resultados.


## Evaluando el rendimiento del modelo

Ahora podemos evaluar el rendimiento de nuestro modelo en el conjunto de datos de prueba. El bucle de evaluación es bastante similar al bucle de entrenamiento, pero no debemos olvidar cambiar el modelo al modo de evaluación llamando a `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Conclusión

En esta unidad, hemos visto lo sencillo que es tomar un modelo de lenguaje preentrenado de la biblioteca **transformers** y adaptarlo a nuestra tarea de clasificación de texto. De manera similar, los modelos BERT pueden utilizarse para la extracción de entidades, respuesta a preguntas y otras tareas de PLN.

Los modelos de transformadores representan el estado del arte actual en PLN, y en la mayoría de los casos deberían ser la primera solución con la que empieces a experimentar al implementar soluciones personalizadas de PLN. Sin embargo, comprender los principios básicos subyacentes de las redes neuronales recurrentes discutidos en este módulo es extremadamente importante si deseas construir modelos neuronales avanzados.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
